In [ ]:
from scipy import sparse as sp
from matplotlib import pyplot as plt
import numpy as np

default_dtype = np.complex128

In [ ]:
# Copied from 05/lanczos.py
Id = sp.csr_matrix(np.eye(2), dtype=default_dtype)
Sx = sp.csr_matrix([[0., 1.], [1., 0.]], dtype=default_dtype)
Sz = sp.csr_matrix([[1., 0.], [0., -1.]], dtype=default_dtype)
Splus = sp.csr_matrix([[0., 1.], [0., 0.]], dtype=default_dtype)
Sminus = sp.csr_matrix([[0., 0.], [1., 0.]], dtype=default_dtype)


def singlesite_to_full(op, i, L):
    op_list = [Id]*L  # = [Id, Id, Id ...] with L entries
    op_list[i] = op
    full = op_list[0]
    for op_i in op_list[1:]:
        full = sp.kron(full, op_i, format="csr")
    return full


def gen_sx_list(L):
    return [singlesite_to_full(Sx, i, L) for i in range(L)]


def gen_sz_list(L):
    return [singlesite_to_full(Sz, i, L) for i in range(L)]


def gen_hamiltonian(sx_list, sz_list, g, J=1.):
    L = len(sx_list)
    H = sp.csr_matrix((2**L, 2**L), dtype=default_dtype)
    for j in range(L):
        H = H - J *( sx_list[j] * sx_list[(j+1)%L])
        H = H - g * sz_list[j]
    return H

def gen_hamiltonian_L(L: int, g: float, J: float = 1.0) -> sp.csr_matrix:
    return gen_hamiltonian(gen_sx_list(L), gen_sz_list(L), g, J)

In [ ]:
L = 14
g = 1.5
J = 1.0
H = gen_hamiltonian_L(L=L, g=g, J=J)
E0, psi_0 = sp.linalg.eigsh(H, k=1, which="SA")
psi_0 = psi_0.flatten()

In [ ]:
assert np.allclose(np.linalg.norm(psi_0), [1.0])
assert psi_0.shape == (2**L,)

In [ ]:
def i_state(i: int, L: int) -> np.ndarray:
    out = np.empty(L, dtype=bool)
    for j in range(L):
        out[j] = i & (1 << j)
    return out.astype(default_dtype)[::-1]

# test
assert np.all(i_state(int("01011", 2), 5) == np.array([0, 1, 0, 1, 1]))

In [ ]:
psi = np.zeros(L, dtype=default_dtype)
for i, psi_i in enumerate(psi_0):
    psi += psi_i * i_state(i, L)
print(psi)

In [ ]:
psi_ab = np.reshape(psi_0, (2**(L//2), 2**(L//2)))

# check if we can recreate the same psi
psi_test = np.zeros(L, dtype=default_dtype)
for a in range(2**(L//2)):
    for b in range(2**(L//2)):
        psi_test += psi_ab[a, b] * np.concatenate((i_state(a, L//2), i_state(b, L//2)), axis=0)

assert np.allclose(psi, psi_test)
del psi_test

In [ ]:
u, lambdas, vh = np.linalg.svd(psi_ab)

assert np.allclose(psi_ab, u @ np.diag(lambdas) @ vh)

psi_test = np.zeros(u.shape, dtype=default_dtype)
for alpha, lambda_alpha in enumerate(lambdas):
    psi_test += lambda_alpha * u[:, alpha].reshape(-1, 1) @ vh[alpha, :].reshape(1, -1)

assert np.allclose(psi_test, psi_ab)
del psi_test

In [ ]:
def get_lambdas(psi: np.ndarray, L: int) -> np.ndarray:
    _psi = psi.copy()
    if len(_psi.shape) == 1:
        _psi = _psi.reshape((2**(L//2), 2**(L//2))) 
    _, lambdas, _ = np.linalg.svd(_psi)
    return lambdas

In [ ]:
plt.figure()
plt.plot(lambdas)
plt.xlabel("$\\alpha$")
plt.ylabel("$\\lambda_\\alpha$")
plt.title("Schmidt values")
plt.yscale("log")
plt.show()

In [ ]:
def get_random_state(L: int) -> np.ndarray:
    state = np.random.normal(size=(2**L)) + 1j * np.random.normal(size=(2**L))
    state /= np.linalg.norm(state)
    return state

In [ ]:
plt.figure()
plt.plot(lambdas, label="$\\Psi_0$")
plt.plot(get_lambdas(get_random_state(L), L), label="$\\Psi_{rnd}$")
plt.xlabel("$\\alpha$")
plt.ylabel("$\\lambda_\\alpha$")
plt.title("Schmidt values")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
def entanglement_entropy(lambdas: np.ndarray) -> float:
    return - np.sum(lambdas**2 * np.log(lambdas**2))

In [ ]:
def print_entropy(g: float, L: int, psi = None) -> float:
    if psi is None:
        H = gen_hamiltonian_L(L=L, g=g, J=J)
        _, psi = sp.linalg.eigsh(H, k=1, which="SA")
        psi = psi.flatten()
    lambdas = get_lambdas(psi, L)
    S = entanglement_entropy(lambdas)
    print(f"{g = }, {L = }: {S = :.3f}, theoretical for random state: {L/2 * np.log(2) - 0.5 :.3f}")
    return S

In [ ]:
Ls = [6, 8, 10, 12, 14, 16, 18]
S_g15 = []
S_g10 = []
S_g05 = []
S_rnd = []
for L in Ls:
    S_g15.append(print_entropy(g=1.5, L=L))
    S_g10.append(print_entropy(g=1.0, L=L))
    S_g05.append(print_entropy(g=0.5, L=L))
    S_rnd.append(print_entropy(g="r", L=L, psi=get_random_state(L)))

In [ ]:
plt.figure()
plt.title("Entanglement entropy")
plt.plot(Ls, S_g15, label="g = 1.5")
plt.plot(Ls, S_g10, label="g = 1.0")
plt.plot(Ls, S_g05, label="g = 0.5")
plt.plot(Ls, S_rnd, label="Random")
plt.plot(Ls, [L/2 * np.log(2) - 0.5 for L in Ls], label="Theory")
plt.legend()
plt.show()

In [ ]:
# I want to see how close we get to the original state with increasing amount of lambdas
from tqdm import tqdm
L = 18
g = 1.5
J = 1.0
H = gen_hamiltonian_L(L=L, g=g, J=J)
E0, psi_0 = sp.linalg.eigsh(H, k=1, which="SA")
psi_ab = psi_0.reshape(2**(L//2), 2**(L//2))

u, s, vh = np.linalg.svd(psi_ab)
norms = []
psi_test = np.zeros(u.shape, dtype=default_dtype)
for alpha in tqdm(range(s.shape[0])):
    psi_test += s[alpha] * u[:, alpha].reshape(-1, 1) @ vh[alpha, :].reshape(1, -1)
    norm = np.linalg.norm(psi_test - psi_ab)
    norms.append(norm)


plt.figure()
plt.plot(norms)
plt.yscale("log")
plt.show()